In [1]:
from google.colab import files
file=files.upload()

Saving winequalityN - winequalityN.csv to winequalityN - winequalityN.csv


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
wine=pd.read_csv("winequalityN - winequalityN.csv")

In [ ]:
wine.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6497 entries, 0 to 6496
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   type                  6497 non-null   object 
 1   fixed acidity         6487 non-null   float64
 2   volatile acidity      6489 non-null   float64
 3   citric acid           6494 non-null   float64
 4   residual sugar        6495 non-null   float64
 5   chlorides             6495 non-null   float64
 6   free sulfur dioxide   6497 non-null   float64
 7   total sulfur dioxide  6497 non-null   float64
 8   density               6497 non-null   float64
 9   pH                    6488 non-null   float64
 10  sulphates             6493 non-null   float64
 11  alcohol               6497 non-null   float64
 12  quality               6497 non-null   int64  
dtypes: float64(11), int64(1), object(1)
memory usage: 660.0+ KB


In [ ]:
wine['type'].unique()

array(['white', 'red'], dtype=object)

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
Oe = OrdinalEncoder()
wine['type'] = Oe.fit_transform(wine[['type']])

In [ ]:
wine.isnull().sum()

,0
type,0
fixed acidity,10
volatile acidity,8
citric acid,3
residual sugar,2
chlorides,2
free sulfur dioxide,0
total sulfur dioxide,0
density,0
pH,9


In [ ]:
wine[wine['fixed acidity'].isnull()]=np.mean(wine['fixed acidity'])
wine[wine['volatile acidity'].isnull()]=np.mean(wine['volatile acidity'])
wine[wine['citric acid'].isnull()]=np.mean(wine['citric acid'])
wine[wine['residual sugar'].isnull()]=np.mean(wine['residual sugar'])
wine[wine['chlorides'].isnull()]=np.mean(wine['chlorides'])
wine[wine['pH'].isnull()]=np.mean(wine['pH'])
wine[wine['sulphates'].isnull()]=np.mean(wine['sulphates'])

<ipython-input-11-5a580c87d89e>:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '7.2165793124710955' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  wine[wine['fixed acidity'].isnull()]=np.mean(wine['fixed acidity'])


In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
previous_imputed = None
for i in range(1, 21):
    imputer = IterativeImputer(max_iter=i, random_state=42)
    wine_imputed=pd.DataFrame(imputer.fit_transform(wine),columns=wine.columns)
    if previous_imputed is not None:
        difference = np.abs(wine_imputed - previous_imputed).max()
        max_difference = difference.max()
        print(f"Iteration {i}: Max change in imputed values = {max_difference:.6f}")
        if max_difference < 1e-3:
            print(f"Converged at iteration {i}")
            break
    previous_imputed = wine_imputed.copy()

Iteration 2: Max change in imputed values = 0.000000
Converged at iteration 2


In [ ]:
X=wine_imputed.drop('quality',axis=1)
y=wine_imputed['quality']

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [ ]:
x_train.shape

(5197, 12)

In [ ]:
y_train

,quality
1916,5.0
947,5.0
877,6.0
2927,7.0
6063,5.0
...,...
3772,6.0
5191,6.0
5226,6.0
5390,7.0


In [ ]:
x_train

,type,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
1916,1.0,6.6,0.25,0.36,8.1,0.045,54.0,180.0,0.99580,3.08,0.42,9.2
947,1.0,8.5,0.16,0.35,1.6,0.039,24.0,147.0,0.99350,2.96,0.36,10.0
877,1.0,6.0,0.28,0.34,1.6,0.119,33.0,104.0,0.99210,3.19,0.38,10.2
2927,1.0,7.0,0.31,0.31,9.1,0.036,45.0,140.0,0.99216,2.98,0.31,12.0
6063,0.0,8.5,0.44,0.50,1.9,0.369,15.0,38.0,0.99634,3.01,1.10,9.4
...,...,...,...,...,...,...,...,...,...,...,...,...
3772,1.0,6.3,0.24,0.29,13.7,0.035,53.0,134.0,0.99567,3.17,0.38,10.6
5191,0.0,6.9,0.36,0.25,2.4,0.098,5.0,16.0,0.99640,3.41,0.60,10.1
5226,0.0,13.4,0.27,0.62,2.6,0.082,6.0,21.0,1.00020,3.16,0.67,9.7
5390,0.0,8.9,0.40,0.51,2.6,0.052,13.0,27.0,0.99500,3.32,0.90,13.4


In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error
for i in range(1,13):
  pca=PCA(n_components=i)
  x_train_tf=pca.fit_transform(x_train)
  knn=KNeighborsRegressor()
  knn.fit(x_train_tf,y_train)
  x_test_tf=pca.transform(x_test)
  y_pred=knn.predict(x_test_tf)
  print(mean_squared_error(y_test, y_pred)) # Example using MSE

0.8130482291977207
0.771867333116703
0.691079433224
0.6726018967693396
0.622125587070154
0.6216640486086155
0.620925587070154
0.6209871255316924
0.6179409716855385
0.6182178947624616
0.6167102024547693
0.6167102024547693


In [ ]:
#Data Visualisation

In [ ]:
x_train_trf=PCA(n_components=2).fit_transform(x_train)


In [ ]:
import plotly.express as px
y_train_trf=y_train.astype(str)
px.scatter(x=x_train_trf[:,0],y=x_train_trf[:,1],color=y_train_trf,color_discrete_sequence=px.colors.qualitative.G10).show()



In [ ]:
x_train_trf=PCA(n_components=3).fit_transform(x_train)

In [ ]:
(px.scatter_3d(x=x_train_trf[:,0],y=x_train_trf[:,1],z=x_train_trf[:,2],color=y_train_trf)).update_layout(margin=dict(l=0,r=0,b=0,t=0)).show()